In [2]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [3]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [5]:

#file_path = "file://<pwd>/Dataset/Complete_Shakespeare.txt"
file_path = "file:////home/talentum/shared/4_AdvancedRddActions/Dataset/constitution.txt"

# Create a baseRDD from the file path
baseRDD = sc.textFile(file_path)

# Split the lines of baseRDD into words
splitRDD = baseRDD.flatMap(lambda x: x.split(' '))

# Removing blank space
splitRDD_no_stop = splitRDD.filter(lambda x: x != '')

# Create a tuple of the word and 1 
splitRDD_no_stop_words = splitRDD_no_stop.map(lambda w: (w, 1))

# Count of the number of occurences of each word
resultRDD = splitRDD_no_stop_words.reduceByKey(lambda x, y: x + y)

# Display the first 10 words and their frequencies
for word in resultRDD.take(10):
    print(word)

print()
# Swap the keys and values 
resultRDD_swap = resultRDD.map(lambda x: (x[1], x[0]))

# Sort the keys in descending order
resultRDD_swap_sort = resultRDD_swap.sortByKey(ascending=False)

# Show the top 10 most frequent words and their frequencies
for word in resultRDD_swap_sort.take(10):
    print("{} has {} counts". format(word[1], word[0]))


('of', 493)
('United', 85)
('in', 137)
('Order', 1)
('form', 1)
('more', 10)
('perfect', 1)
('Union,', 4)
('establish', 4)
('Tranquility,', 1)

the has 662 counts
of has 493 counts
shall has 293 counts
and has 256 counts
to has 183 counts
be has 178 counts
or has 157 counts
in has 137 counts
by has 100 counts
a has 94 counts


In [7]:
file_path = "file:////home/talentum/shared/4_AdvancedRddActions/Dataset/constitution.txt"

# Create a baseRDD from the file path
baseRDD = sc.textFile(file_path)

# Convert line to words
rdd_words = baseRDD.flatMap(lambda line : line.split(' '))

# Converting words to tuple
rdd_tup = rdd_words.map(lambda word: (word , 1))

# Counting frequency of each word
rdd_reduce = rdd_tup.reduceByKey(lambda x, y : x + y)

# Swaping the values in tuple
rdd_swap = rdd_reduce.map(lambda tup : (tup[1], tup[0]))

# Sorting the values
rdd_sort = rdd_swap.sortByKey(ascending = False).take(10)
rdd_sort

[(812, ''),
 (662, 'the'),
 (493, 'of'),
 (293, 'shall'),
 (256, 'and'),
 (183, 'to'),
 (178, 'be'),
 (157, 'or'),
 (137, 'in'),
 (100, 'by')]

In [8]:
#same code in single line (Chaining)

file_path = "file:////home/talentum/shared/4_AdvancedRddActions/Dataset/constitution.txt"


sc.textFile(file_path) \
.flatMap(lambda line : line.split(' ')) \
.map(lambda word: (word , 1)) \
.reduceByKey(lambda x, y : x + y) \
.map(lambda tup : (tup[1], tup[0])) \
.sortByKey(ascending = False).take(10) 

[(812, ''),
 (662, 'the'),
 (493, 'of'),
 (293, 'shall'),
 (256, 'and'),
 (183, 'to'),
 (178, 'be'),
 (157, 'or'),
 (137, 'in'),
 (100, 'by')]